In [ ]:
!nvidia-smi

Thu Jul 13 12:58:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git sentencepiece git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


In [ ]:
!accelerate env

2023-07-13 12:59:21.492747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.21.0.dev0
- Platform: Linux-5.15.109+-x86_64-with-glibc2.31
- Python version: 3.10.12
- Numpy version: 1.22.4
- PyTorch version (GPU?): 2.0.1+cu118 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 12.68 GB
- GPU type: Tesla T4
- `Accelerate` default config:
	Not found


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
!rm -rf /content/offload_folder
!mkdir /content/offload_folder

In [ ]:
# Load 45 GB Model  -> 11 Billion Model
model = AutoModelForSeq2SeqLM.from_pretrained(
  "bigscience/T0pp",
  revision="sharded",
  device_map="auto",
  offload_state_dict=True,
  max_memory={0: "11GIB", "cpu": "11GIB"},
  #max_memory={"cpu": "11GIB"},
  offload_folder="/content/offload_folder",
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Load 45 GB Model  -> 11 Billion Model
model = AutoModelForSeq2SeqLM.from_pretrained(
  "bigscience/T0pp",
  revision="sharded",
  device_map="auto",
  offload_state_dict=True,
  max_memory={0: "12GIB", "cpu": "22GIB"},
  #max_memory={"cpu": "11GIB"},
  offload_folder="/content/offload_folder",
)

In [ ]:
text = "Classify each of the following sentences as high price,  coverage, claim or refund, vehicle change, customer service, agent service, broker service, bank service, cancelation process, or renewal . sentence: Without asking me they changed my agent/mediator. I want to make it clear that the client decides who they want as a mediator  Label:"
inputs = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

Hmmm, Good point, but in Colab there is free 12GB of memory which should fit each shard file.

More details :
Colab CPU doesn't work even there is enough memory to load the shard file:
Memory: 12GB Free Memory .
HDD: 140GB Free Disk Space
Shard File: 10GB
Max Memory: 11GB

#```
 Load 45 GB Model  -> 11 Billion Model
model = AutoModelForSeq2SeqLM.from_pretrained(
  "bigscience/T0pp",
  revision="sharded",
  device_map="auto",
  offload_state_dict=True,
  #max_memory={0: "11GIB", "cpu": "11GIB"},
  max_memory={"cpu": "11GIB"},
  offload_folder="/content/offload_folder",
)
```

```
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
[<ipython-input-7-6a6a84c1e707>](https://localhost:8080/#) in <module>()
      7   #max_memory={0: "4GIB", "cpu": "4GIB"},
      8   max_memory={"cpu": "11GIB"},
----> 9   offload_folder="/content/offload_folder",
     10 )

2 frames
[/usr/local/lib/python3.7/dist-packages/transformers/models/auto/auto_factory.py](https://localhost:8080/#) in from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs)
    444         elif type(config) in cls._model_mapping.keys():
    445             model_class = _get_model_class(config, cls._model_mapping)
--> 446             return model_class.from_pretrained(pretrained_model_name_or_path, *model_args, config=config, **kwargs)
    447         raise ValueError(
    448             f"Unrecognized configuration class {config.__class__} for this kind of AutoModel: {cls.__name__}.\n"

[/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py](https://localhost:8080/#) in from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs)
   2141         # Dispatch model with hooks on all devices if necessary
   2142         if device_map is not None:
-> 2143             dispatch_model(model, device_map=device_map, offload_dir=offload_folder)
   2144
   2145         if output_loading_info:

[/usr/local/lib/python3.7/dist-packages/accelerate/big_modeling.py](https://localhost:8080/#) in dispatch_model(model, device_map, main_device, state_dict, offload_dir, offload_buffers, preload_module_classes)
    210
    211     if main_device is None:
--> 212         main_device = [d for d in device_map.values() if d not in ["cpu", "disk"]][0]
    213
    214     cpu_modules = [name for name, device in device_map.items() if device == "cpu"]

IndexError: list index out of range
```


Colab GPU doesn't work even there is enough memory to load the shard file:
GPU: V100 16 GB Free Memory.
Memory: 12GB Free Memory .
HDD: 140GB Free Disk Space
Shard File: 10GB
Max Memory: 11GB

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

In [ ]:
# Load 45 GB Model  -> 11 Billion Model
model = AutoModelForSeq2SeqLM.from_pretrained(
  "google/flan-t5-xxl",
  revision="main",
  device_map="auto",
  offload_state_dict=True,
  max_memory={0: "12GIB", "cpu": "22GIB"},
  #max_memory={"cpu": "11GIB"},
  offload_folder="/content/offload_folder",
)

In [ ]:
text = "Classify each of the following sentences as brand , culture , digital , diversity , economic , environmental , financial , organization , partnership , strategic , or work . " + \
"sentence: In an interview with @€uroamSonntag, Board Member Jacqueline Hunt emphasized that despite market uncertainty, 2021 directionally looks more optimistic than months ago. " + \
"Label: "
inputs = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

In [ ]:
# Load 45 GB Model  -> 11 Billion Model
model = AutoModelForSeq2SeqLM.from_pretrained(
  "google/flan-t5-xxl",
  revision="main",
  device_map="auto",
  offload_state_dict=True,
  max_memory={0: "15GIB", "cpu": "70GIB"},
  #max_memory={"cpu": "11GIB"},
  offload_folder="/home/azure/ahmed/offload_folder",
)